In [11]:
import astroprov
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils import DAOStarFinder
from photutils import CircularAperture
from photutils import aperture_photometry
#import subproc

import numpy as np
from astropy import wcs
from astropy.io import fits
import sys

from astroquery.simbad import Simbad

import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table, vstack
from astropy.table import Column

import collections
import subprocess
from astropy.io import ascii


In [2]:
def alterDefault(defaultDir,valList,attributeList,catName):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    with open(defaultDir+'/default.sex','r') as f:
        endData = f.read()
    cname = 'CATALOG_NAME'
    nameLoc = endData.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    #print(newData)
    with open(defaultDir+'/default.sex','w') as f:
        f.write(newData)

In [3]:
def sextractor(sextractoryDir,imagename,cataloguename,imageDir='/home/mj1e16/keplerImages/'):
    os.chdir(sextractoryDir)
    subprocess.call(['sex',imageDir+imagename])
    assoc = Table.read(cataloguename,format='ascii.sextractor')

#     os.chdir(imageDir)
#     hdu_list = fits.open(imagename) # open image header
#     imagedata = (hdu_list[0].data) 
    
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (assoc['X_IMAGE'], assoc['Y_IMAGE'])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
#     print(len(assoc))
#     mags = assoc['MAG_APER']
#     plt.hist(mags,bins='auto')
#     plt.show()
    return assoc

In [4]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
#         print(fileName)
#         print(os.listdir('.'))

In [9]:
def load_wcs_from_file(filename,pixCoord):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[0].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [10]:
def queryObjectRegion(outputTable,imageName,inCat,searchRadius=5*u.arcsecond,imDir='/home/mj1e16/simages/'):
    imageCoords = []
    r = searchRadius
    
    if inCat == 'dao':
        xlabel = 'xcentroid'
        ylabel = 'ycentroid'
        label = 'id'
        
    if inCat == 'sex':
        xlabel = 'X_IMAGE'
        ylabel = 'Y_IMAGE'
        label = 'NUMBER'
    flux = []   
    tableLength = outputTable[label][-1]
    for x in range(tableLength):
        xpos = outputTable[xlabel][x]
        ypos = outputTable[ylabel][x]
        flux.append(outputTable['FLUX_MAX'][x])
        if 20 < ypos < 1040:
            if 12 < xpos < 1110:
                imageCoords.append([xpos,ypos])
                # this may need to change per image but I hope not

    coords = load_wcs_from_file(imDir+imageName,imageCoords)
    customSimbad = Simbad()
    customSimbad.add_votable_fields('ra(d)','dec(d)','otype')
    customSimbad.remove_votable_fields('coordinates')
            
    result_table = Table([['-'],[0],[0],['-'],[0],[0],[0],['-']],names=('MAIN_ID','RA_d','DEC_d','OTYPE','xcentroid','ycentroid','flux','IMAGE'),dtype=('object','float64','float64','object','float64','float64','float64','object'))

    for x in range(len(imageCoords)):
        try:
            c = SkyCoord(coords[x][0],coords[x][1],frame='icrs',unit='deg')
            rt = customSimbad.query_region(c,radius=r)
            rowlist = []
            for y in range(len(rt[0])):
                rowlist.append(rt[0][y])
            rowlist.append(imageCoords[x][0])
            rowlist.append(imageCoords[x][1])
            rowlist.append(flux[x])
            rowlist.append(imageName)
            result_table.add_row(rowlist)

        except:
            result_table.add_row(['N/A',[0],[0],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],imageName])
    
#     hdu_list = fits.open(imageName)
#     imagedata = (hdu_list[0].data) 
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (outputTable[xlabel], outputTable[ylabel])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
    return result_table

In [7]:
def runAndShow(image,cat,settings=[10,5,64,3],imDir='/home/mj1e16/simages/',sexDir='/home/mj1e16/sextractor/sextractor-master/config/'):
    alterDefault(sexDir,settings,['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE'],catName)
    tab = sextractor(sexDir,image,cat,imageDir=imDir)
    pan = tab.to_pandas()
    pan = pan[pan['MAG_APER'] != 99.0]
    pan = pan[pan['ELLIPTICITY'] < np.median(pan['ELLIPTICITY'])]
    pan = pan[pan['ELONGATION'] < np.median(pan['ELONGATION'])]

#     pan = pan[pan['X_IMAGE'] < 610.]
#     pan = pan[pan['X_IMAGE'] > 570.]
#     pan = pan[pan['Y_IMAGE'] < 300.]
#     pan = pan[pan['Y_IMAGE'] > 250.]
    tab = Table.from_pandas(pan)
    print(len(tab))
    searchTab = queryObjectRegion(tab,image,'sex',imDir=imDir)
#     makeDS9RegFile([tab],'/home/mj1e16/simages/autosextest','sex')
#     subprocess.call(['ds9',imDir+image,'-regions','/home/mj1e16/simages/autosextest.reg'])
    return searchTab

In [ ]:
sexDir = '/home/mj1e16/sextractor/sextractor-master/config/'
catName = 'testauto.cat'

In [8]:
outTab = runAndShow('diff_44_1.fits',catName,imDir=)

399
/home/mj1e16/simages/autosextest.reg
['gauss_4.0_7x7.conv', 'default.paramold', 'tophat_3.0_3x3.conv', 'tophat_1.5_3x3.conv', 'gauss_3.0_5x5.conv', 'tophat_4.0_5x5.conv', 'README', '.ipynb_checkpoints', 'tophat_2.0_3x3.conv', 'gauss_2.5_5x5.conv', 'tophat_5.0_5x5.conv', 'mexhat_2.5_7x7.conv', 'gauss_1.5_3x3.conv', 'tophat_2.5_3x3.conv', 'block_3x3.conv', 'test.cat', 'keplerPhotPlot.ipynb', 'mexhat_2.0_7x7.conv', 'fits.png', 'default.conv', 'default.param', 'testauto.cat', 'mexhat_4.0_9x9.conv', 'default.sex', 'gauss_2.0_3x3.conv', 'mexhat_1.5_5x5.conv', 'gauss_3.0_7x7.conv', 'gauss_5.0_9x9.conv', 'mexhat_5.0_11x11.conv', 'daoStarFinder1552000906.550.935156930732.json', 'default.nnw', 'mexhat_3.0_9x9.conv', 'gauss_2.0_5x5.conv', 'autosextest0.reg']


In [96]:
imdir = 
dirlist = os.listdir(imdir)
dirlist = [x for x in dirlist if x[-4:] =='fits'] # check
print(dirlist)


for im in range(len(dirlist)):
    if im == 0:
        outTab = runAndShow('diff_44_1.fits',catName,imDir=imdir)
    else:
        newTab = runAndShow('diff_44_1.fits',catName,imDir=imdir)
        outTab = vstack([outTab,newTab])
        
df = outTab.to_pandas()
df = df[df['OTYPE'] == 'Object Not Found']
dfsort = df.sort_values(['FLUX_MAX'])

NUMBER,X_IMAGE,Y_IMAGE,ALPHA_J2000,DELTA_J2000,FLUX_APER,FLUXERR_APER,MAG_APER,FLUX_RADIUS,FWHM_IMAGE,BACKGROUND,THRESHOLD,FLUX_MAX,ISOAREA_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,FLAGS,ELONGATION,ELLIPTICITY
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64
2,254.6297,43.2683,292.7373078,44.3123446,-106.6064,8.954877,99.0,7.507,13.65,-0.1794719,20.18585,779.515,80,3.848,2.557,-2.06,2,1.505,0.335
30,717.7286,932.8685,291.1954828,44.223114,-9.318528,8.918401,99.0,0.286,3.13,-0.1700412,20.18585,405.7114,2,0.685,0.416,73.19,0,1.648,0.393
46,163.6603,862.0655,291.7436112,43.7486244,-273.8882,8.945772,99.0,-17.986,0.0,-0.1670337,20.18585,208.6573,4,0.985,0.434,-56.43,3,2.271,0.56
69,852.6743,769.0045,291.2962958,44.4462838,-35.99896,8.945772,99.0,0.396,0.0,-0.1543486,20.18585,409.162,2,0.689,0.413,-66.38,0,1.666,0.4
92,883.8236,680.2496,291.3861721,44.528059,-153.4849,8.963973,99.0,0.416,3.15,-0.1292565,20.18585,452.4327,3,0.694,0.416,-12.23,0,1.667,0.4
100,305.8099,661.6744,291.8872547,43.999534,-4626.514,8.954877,99.0,-252.998,5.74,-0.475937,20.18585,92.96677,11,2.002,0.649,19.51,2,3.084,0.676
105,915.0876,650.7775,291.3986472,44.5746891,-9.960415,8.936658,99.0,0.41,3.57,-0.1809916,20.18585,406.0842,3,0.806,0.455,-38.17,0,1.769,0.435
141,433.6249,528.0815,291.9566424,44.1976388,-15847.33,8.945772,99.0,-19.325,5.47,-0.4902271,20.18585,672.9816,27,2.79,1.026,55.03,2,2.719,0.632
142,428.0,529.0903,291.9599328,44.191783,-524.8409,8.963973,99.0,8.253,9.63,-0.4850797,20.18585,68.86225,14,2.839,0.912,-88.07,2,3.114,0.679


In [69]:
outPan = outTab.to_pandas()
sortTab = outPan.sort_values(['ELONGATION'])


In [70]:
sortTab

,NUMBER,X_IMAGE,Y_IMAGE,ALPHA_J2000,DELTA_J2000,FLUX_APER,FLUXERR_APER,MAG_APER,FLUX_RADIUS,FWHM_IMAGE,BACKGROUND,THRESHOLD,FLUX_MAX,ISOAREA_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,FLAGS,ELONGATION,ELLIPTICITY
1626,2889,407.4532,409.4460,0.0,0.0,1227.86600,9.083923,-7.7229,0.804,0.98,-0.669483,6.13679,366.29710,8,0.870,0.869,-70.58,0,1.001,0.001
2449,4183,480.9913,582.9999,0.0,0.0,147.95830,9.065479,-5.4253,0.419,0.00,-0.420949,6.13679,129.85340,3,0.715,0.714,71.92,0,1.002,0.002
3449,5657,636.6011,667.0096,0.0,0.0,927.34060,9.065479,-7.4181,0.722,1.07,-0.422779,6.13679,390.62750,9,0.852,0.850,48.01,0,1.002,0.002
2948,4872,177.2634,779.3145,0.0,0.0,4380.21600,9.037742,-9.1037,0.820,1.69,-0.285206,6.13679,1184.95400,10,0.921,0.919,65.99,0,1.002,0.002
169,296,952.9039,55.9717,0.0,0.0,11128.91000,9.046997,-10.1161,0.956,2.55,-0.037754,6.13679,2427.59500,12,0.973,0.972,-56.56,0,1.002,0.002
3336,5470,415.5953,684.6047,0.0,0.0,731.91220,9.065478,-7.1611,0.788,0.98,-0.372609,6.13679,252.34390,7,0.866,0.865,56.00,0,1.002,0.002
808,1527,277.9213,219.6775,0.0,0.0,2729.95500,9.065479,-8.5904,0.737,1.34,-0.762254,6.13679,1094.57900,8,0.904,0.902,62.21,3,1.002,0.002
1830,3211,141.9317,445.3230,0.0,0.0,2254.89200,9.037742,-8.3828,1.219,2.12,-0.613897,6.13679,785.11540,13,1.150,1.148,71.16,0,1.002,0.002
3319,5445,364.4733,689.9902,0.0,0.0,669.48400,9.074706,-7.0644,0.734,1.19,-0.445358,6.13679,239.15020,7,0.854,0.852,-37.33,0,1.002,0.002
2299,3959,824.2881,548.0385,0.0,0.0,807.67320,9.065479,-7.2681,0.708,1.15,-0.340390,6.13679,358.38990,8,0.851,0.848,-65.54,0,1.003,0.003


In [99]:
print(np.median(outTab['ELLIPTICITY']))

nan


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2954: RuntimeWarning: Mean of empty slice.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [100]:
print(np.median(outTab['ELONGATION']))
print(np.mean(outTab['ELONGATION']))

nan
nan
